In [5]:
#!pip install nltk
#!pip install scikit-learn
#!pip install pandas

<h4>Importing required packages</h4>

In [6]:
import pandas as pd
import nltk
import re
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
ps = SnowballStemmer(language = "english")
nltk.download("stopwords")

[nltk_data] Downloading package stopwords to /config/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

<h5>Read data</h5>

In [7]:
df = pd.read_csv("/config/workspace/emotions/data/data.csv")


<h4>Function for cleaning text data</h4>

In [8]:
def clean_text(text):
    result = re.sub("[^a-zA-Z]"," ",text)
    result = result.lower()
    result = result.split()
    result = [ps.stem(word) for word in result if word not in set(stopwords.words('english'))]
    result = " ".join(result)
    return result

In [9]:
df['cleaned_text'] = df["text"].apply(clean_text)

In [10]:
df.head()

,text,emotions,cleaned_text
0,i can go from feeling so hopeless to so damned...,sadness,go feel hopeless damn hope around someon care ...
1,im grabbing a minute to post i feel greedy wrong,anger,im grab minut post feel greedi wrong
2,i am ever feeling nostalgic about the fireplac...,love,ever feel nostalg fireplac know still properti
3,i am feeling grouchy,anger,feel grouchi
4,ive been feeling a little burdened lately wasn...,sadness,ive feel littl burden late wasnt sure


<h4>text to numeric</h4>

In [11]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer()

 